#ND

In [ ]:
%pip install gensim nltk scikit-learn

  Using cached pandas-2.2.3-cp310-cp310-macosx_11_0_arm64.whl (11.3 MB)
  Using cached pytz-2025.1-py2.py3-none-any.whl (507 kB)
  Using cached tzdata-2025.1-py2.py3-none-any.whl (346 kB)

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import nltk
# Download tokenizer and dataset

nltk.download('punkt')
nltk.download('movie_reviews')

[nltk_data] Downloading package punkt to /Users/balaji/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     /Users/balaji/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


True

In [5]:
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from nltk.tokenize import word_tokenize
import numpy as np


from nltk.corpus import movie_reviews

# Step 1: Load IMDb-like dataset (from NLTK movie reviews)
data = [(list(movie_reviews.words(fileid)), category)
        for category in movie_reviews.categories()
        for fileid in movie_reviews.fileids(category)]

# Split data into sentences (reviews) and labels
sentences, labels = zip(*data)
print(sentences[0])
print(labels[0])
# Convert labels into binary format
labels = [1 if label == "pos" else 0 for label in labels]

# Tokenize the dataset (already tokenized in NLTK; skip this if necessary)
tokenized_sentences = [sentence for sentence in sentences]

# Step 2: Train Word2Vec model on the tokenized sentences
word2vec_model = Word2Vec(sentences=tokenized_sentences, vector_size=100, window=5, min_count=1, sg=1)

# Function to compute sentence embeddings by averaging word vectors
def get_sentence_embedding(sentence_tokens, model):
    word_vectors = [model.wv[word] for word in sentence_tokens if word in model.wv]
    if len(word_vectors) > 0:
        return np.mean(word_vectors, axis=0)  # Average word vectors
    else:
        return np.zeros(model.vector_size)  # Return a zero vector if no words are in the model

# Step 3: Compute embeddings for all sentences
X = np.array([get_sentence_embedding(tokens, word2vec_model) for tokens in tokenized_sentences])
y = np.array(labels)
print(X[0])
# Step 4: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 5: Train a classifier (Logistic Regression)
classifier = make_pipeline(StandardScaler(), LogisticRegression(C=1))
classifier.fit(X_train, y_train)

# Step 6: Test the classifier
accuracy = classifier.score(X_test, y_test)
print(f"Classifier Accuracy: {accuracy:.2f}")

# Step 7: Predict on new sentences
new_sentences = ["The movie was fantastic and had great visuals",
                 "I hated the plot and the acting was terrible"]
new_tokenized = [word_tokenize(sentence.lower()) for sentence in new_sentences]
new_embeddings = np.array([get_sentence_embedding(tokens, word2vec_model) for tokens in new_tokenized])

# Predict using the trained classifier
predictions = classifier.predict(new_embeddings)
for sentence, prediction in zip(new_sentences, predictions):
    sentiment = "Positive" if prediction == 1 else "Negative"
    print(f"Sentence: {sentence} -> Sentiment: {sentiment}")



['plot', ':', 'two', 'teen', 'couples', 'go', 'to', 'a', 'church', 'party', ',', 'drink', 'and', 'then', 'drive', '.', 'they', 'get', 'into', 'an', 'accident', '.', 'one', 'of', 'the', 'guys', 'dies', ',', 'but', 'his', 'girlfriend', 'continues', 'to', 'see', 'him', 'in', 'her', 'life', ',', 'and', 'has', 'nightmares', '.', 'what', "'", 's', 'the', 'deal', '?', 'watch', 'the', 'movie', 'and', '"', 'sorta', '"', 'find', 'out', '.', '.', '.', 'critique', ':', 'a', 'mind', '-', 'fuck', 'movie', 'for', 'the', 'teen', 'generation', 'that', 'touches', 'on', 'a', 'very', 'cool', 'idea', ',', 'but', 'presents', 'it', 'in', 'a', 'very', 'bad', 'package', '.', 'which', 'is', 'what', 'makes', 'this', 'review', 'an', 'even', 'harder', 'one', 'to', 'write', ',', 'since', 'i', 'generally', 'applaud', 'films', 'which', 'attempt', 'to', 'break', 'the', 'mold', ',', 'mess', 'with', 'your', 'head', 'and', 'such', '(', 'lost', 'highway', '&', 'memento', ')', ',', 'but', 'there', 'are', 'good', 'and', 'ba